# Mapeamento

O elastisearch possui um processo interno que identifica o tipo de dado de cada cháve que é salva em um determinado indíce (processo chamado indice). Neste notebook, será discutido como o elastisearch mapea cada tipo de dado automaticamente e como fazer o processo manual , útil em diversas situações. 

Para mostrar essa funcionalidade, utilizaremos o seguinte dicionário que representa dados de uma música

Para melhor detalhes , acesse o [link](https://www.elastic.co/docs/reference/elasticsearch/mapping-reference/field-data-types)


In [1]:
import datetime
import base64
letra_musica = {
    "titulo": "Meu Novo Mundo",
    "autor": "Charlie Brown Jr",
    "letra": "Como se o silêncio dissesse tudo  \n  Um sentimento bom que me leva pra outro mundo  \n  A vontade de te ver já é maior que tudo  \n  Não existem distâncias no meu novo mundo  \n    \n  Tipo coisas da sétima arte  \n  Aconteceu sem que eu imaginasse  \n    \n  Sonho de consumo cantar na sua festa  \n  Vem dançar comigo  \n  Aproveita e me sequestra  \n  Amor vagabundo, intenso ou muita pressa  \n  Não sei como termina mas sei como começa  \n    \n  Fiz essa canção pra dizer algumas coisas  \n  Cuidado com o destino  \n  Ele brinca com as pessoas  \n  Tipo uma foto com sorriso inocente  \n  Mas a vida tinha um plano e separou a gente  \n    \n  Mas se quem eu amo tem amor por mim  \n  Se quem eu amo tem amor por mim  \n  Eu sei que ainda estamos muito longe do fim  \n    \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo  \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo  \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo  \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo  \n    \n  Fiz essa canção pra dizer algumas coisas  \n  Cuidado com o destino  \n  Ele brinca com as pessoas  \n  Tipo uma foto com sorriso inocente  \n  Mas a vida tinha um plano e separou a gente  \n    \n  Mas se quem eu amo tem amor por mim  \n  Se quem eu amo tem amor por mim  \n  Eu sei que ainda estamos muito longe do fim  \n    \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo  \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo  \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo  \n  A vontade de te ver já é maior que tudo  \n  E não existem distâncias no meu novo mundo",
    "compositor": "Compositores: Alexandre Magno Abrao (UBC), Thiago Raphael Castanho (ABRAMUS)",
    "editor": "Editores: Sony Music (UBC), Digital Grooves Records (ABRAMUS)",
    "url": "https://www.vagalume.com.br/charlie-brown-jr/meu-novo-mundo.html",
    "administracao": "Administração: Sony Music Publishing (UBC)",
    "ano": 2013,
    "extracao": "2025-06-03",
    "extracao_mes":"2025-06",
    "extracao_exata": "2025-06-03 12:00:00",
    "genero":"rock",
    "pontuacao_letras":4.5,
    "pontuacao_vagalume":4,
    "url_letras":"https://www.letras.mus.br/charlie-brown-jr/meu-novo-mundo/",
    "comentarios":[
        {
            "autor":"gal tropical",
            "data":datetime.datetime(2023, 10, 1, 12, 30),
            "titulo":"Tipo coisas da sétima arte",
            "texto":"Essa música é uma viagem no tempo, me faz lembrar de momentos incríveis com pessoas especiais. A letra é profunda e cheia de significado, como se cada verso fosse um pedaço da minha própria história. Charlie Brown Jr sempre soube como tocar o coração da gente.",
            "ip_comentarista":"192.168.1.10"
        }
    ],
    "bytes_mp4": base64.b64encode(b'some binary data').decode('utf-8'),
    "musica_tocada_radio": False
}

Criando indice e salvando a música

In [2]:
# Criando o conector com elasticsearch
# Modificar a variável PASSWORD para a senha configurado no .env
from elasticsearch import Elasticsearch

HOST = 'http://localhost:9200'
USERNAME = 'elastic'
PASSWORD = 'test10'

client = Elasticsearch(
    HOST,
    basic_auth=(USERNAME,PASSWORD)
)


In [3]:
INDEX_MAPPING_MUSICA = 'mapeamento_musica'

In [4]:
if client.indices.exists(index=INDEX_MAPPING_MUSICA):
    client.indices.delete(index=INDEX_MAPPING_MUSICA)
client.indices.create(index=INDEX_MAPPING_MUSICA)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'mapeamento_musica'})

In [5]:
#Salvando o primeiro objeto e verificando o mapeamento
client.index(index=INDEX_MAPPING_MUSICA, document=letra_musica)

ObjectApiResponse({'_index': 'mapeamento_musica', '_id': '4bOSS5cBUW64_IbYCx32', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [6]:
mapping_musica = client.indices.get_mapping(index=INDEX_MAPPING_MUSICA)

In [7]:
mapping_musica.body['mapeamento_musica']['mappings']

{'properties': {'administracao': {'type': 'text',
   'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
  'ano': {'type': 'long'},
  'autor': {'type': 'text',
   'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
  'bytes_mp4': {'type': 'text',
   'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
  'comentarios': {'properties': {'autor': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'data': {'type': 'date'},
    'ip_comentarista': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'texto': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'titulo': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
  'compositor': {'type': 'text',
   'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
  'editor': {'type': 'text',
   'fields': {'keyword': {'type': 'keyword', 'ign

### Campo texto

Ententendo como cada chave foi identificada no elastisearch.

Começando com as chaves do tipo texto, como "administracao","compositor" e "letra"

In [8]:
# Olhando a chave "compositor"
mapping_musica.body['mapeamento_musica']['mappings']['properties']['compositor']

{'type': 'text',
 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}

A chave 'compositor' foi identificada como sendo do tipo texto (text). No entanto, o Elasticsearch também criou automaticamente um field chamado 'compositor.keyword', mapeando-o como do tipo palavra-chave (keyword).

O 'field'  permite que uma mesma chave seja representada com mais de um tipo de dado. No caso, 'compositor' é do tipo texto, enquanto 'compositor.keyword' é do tipo palavra-chave. Isso possibilita diferentes formas de consulta sobre o mesmo valor, sem a necessidade de duplicar os dados.

A principal diferença entre os tipos text e keyword está na forma como as buscas são realizadas:

1. O tipo text permite pesquisas por trechos ou palavras dentro do conteúdo, utilizando os algoritmos de análise textual do Elasticsearch.

2. Já o tipo keyword é usado para buscas exatas, sem análise ou divisão do conteúdo.


Por default, todos os campos do tipo texto/strings são mapeados como texto e criado um field do tipo keyword, inclusive ip e url



In [9]:
# Olhando a chave "letra" 
mapping_musica.body['mapeamento_musica']['mappings']['properties']['letra']

{'type': 'text',
 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}

Fazendo pesquisa pelo o texto e pela palavra-chave para mostrar a difença

In [10]:
#Pesquisando por autor
search_query = {
    "query": {
        "match": {
            "autor": "Charlie"
        }
    }
}
response = client.search(index=INDEX_MAPPING_MUSICA, body=search_query)

In [11]:
response.body

{'took': 65,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.2876821,
  'hits': [{'_index': 'mapeamento_musica',
    '_id': '4bOSS5cBUW64_IbYCx32',
    '_score': 0.2876821,
    '_ignored': ['comentarios.texto.keyword', 'letra.keyword'],
    '_source': {'titulo': 'Meu Novo Mundo',
     'autor': 'Charlie Brown Jr',
     'letra': 'Como se o silêncio dissesse tudo  \n  Um sentimento bom que me leva pra outro mundo  \n  A vontade de te ver já é maior que tudo  \n  Não existem distâncias no meu novo mundo  \n    \n  Tipo coisas da sétima arte  \n  Aconteceu sem que eu imaginasse  \n    \n  Sonho de consumo cantar na sua festa  \n  Vem dançar comigo  \n  Aproveita e me sequestra  \n  Amor vagabundo, intenso ou muita pressa  \n  Não sei como termina mas sei como começa  \n    \n  Fiz essa canção pra dizer algumas coisas  \n  Cuidado com o destino  \n  Ele brinca com as pessoas  \n  Ti

In [12]:
#Pesquisando por autor.keyword
search_query_keyword = {
    "query": {
        "match": {
            "autor.keyword": "Charlie"
        }
    }
}

response_keyword = client.search(index=INDEX_MAPPING_MUSICA, body=search_query_keyword)

In [13]:
response_keyword.body

{'took': 11,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

### Campo de data

Os campos relacionados a datas, como as chaves "extracao" e "comentarios.data", foram mapeados como tipo data (date). Isso ocorreu mesmo com "extracao" sendo originalmente um texto e "comentarios.data" do tipo datetime. Qualquer valor textual no formato 'yyyy-MM-dd' ou 'yyyy-MM' é automaticamente interpretado como uma data.

Por outro lado, o campo "ano" não foi mapeado como data, mas sim como número.

O mapeamento desses campos como datas permite realizar pesquisas com operadores como maior ou menor que uma data específica, ou mesmo filtrar por um dia exato. Veja o exemplo abaixo:

In [14]:
#Pesquisando por uma música com dia de extração específico
search_query = {
    "query": {
        "script": {
            "script": {
                "source": "doc['extracao'].value.dayOfMonth == 3",
                "lang": "painless"
                }
        }
    }
}

response = client.search(index=INDEX_MAPPING_MUSICA, body=search_query)

In [15]:
response.body

{'took': 962,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'mapeamento_musica',
    '_id': '4bOSS5cBUW64_IbYCx32',
    '_score': 1.0,
    '_ignored': ['comentarios.texto.keyword', 'letra.keyword'],
    '_source': {'titulo': 'Meu Novo Mundo',
     'autor': 'Charlie Brown Jr',
     'letra': 'Como se o silêncio dissesse tudo  \n  Um sentimento bom que me leva pra outro mundo  \n  A vontade de te ver já é maior que tudo  \n  Não existem distâncias no meu novo mundo  \n    \n  Tipo coisas da sétima arte  \n  Aconteceu sem que eu imaginasse  \n    \n  Sonho de consumo cantar na sua festa  \n  Vem dançar comigo  \n  Aproveita e me sequestra  \n  Amor vagabundo, intenso ou muita pressa  \n  Não sei como termina mas sei como começa  \n    \n  Fiz essa canção pra dizer algumas coisas  \n  Cuidado com o destino  \n  Ele brinca com as pessoas  \n  Tipo uma foto

### Campo de número

Os campos "ano", "pontuacao_letras" e "pontuacao_vagalume" foram mapeados como número não inteiros (padrão do elastisearch)

In [16]:
mapping_musica.body['mapeamento_musica']['mappings']['properties']['ano']

{'type': 'long'}

## Mapeamento Manual

Muitas vezes, o mapeamento automático do **Elasticsearch** não é o mais adequado e pode causar problemas de desempenho à medida que o banco de dados cresce.

Abaixo estão alguns problemas comuns ao deixar o mapeamento automático:

1. **Uso ineficiente de recursos** ao mapear um campo de texto como *keyword* (palavra-chave) quando não se pretende realizar buscas exatas — por exemplo, o campo **"letra"**.

2. **Mapeamento inadequado para buscas exatas**, ao tratar como texto campos onde a correspondência exata é mais relevante — como o campo **"genero"**.

3. **Mapeamento incorreto de datas como números**, como ocorre com o campo **"ano"**.

4. **Mapeamento incorreto de endereços IP**, resultando em perda de funcionalidade em consultas específicas para esse tipo de dado.


O elastisearch permite criar o mapeamento antes que de carregar o primeiro objeto no indice, evitando assim que as chaves ja mapeadas não sejam interpretadas de forma diferença da informada

In [29]:
#Criando o mapeamento 

novo_mapeamento = {
    "properties": {
        "titulo": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
        "autor": {"type": "keyword"},
        "letra": {"type": "text"},
        "compositor": {"type": "keyword"},
        "editor": {"type": "text"},
        "url": {"type": "keyword"},
        "administracao": {"type": "text"},
        "ano": {"type": "date", "format": "yyyy"}, #mapeando como data , no formato yyyy
        "extracao": {"type": "date", "format": "yyyy-MM-dd"},
        "extracao_mes": {"type": "date","format": "yyyy-MM"}, #mapeando como data, no formato yyyy-MM
        "extracao_exata": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"}, #mapeando como data, no formato yyyy-MM-dd HH:mm:ss
        "genero": {"type": "keyword"},
        "pontuacao_letras": {"type": "float"},
        "pontuacao_vagalume": {"type": "float"},
        "url_letras": {"type": "keyword"},
        "comentarios": {
            "type": "nested",
            "properties": {
                "autor": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
                "data": {"type": "date"}, # mapeando como data, no formato ISO 8601
                "titulo":{"type":"text",    "fields": {"keyword": {"type": "keyword"}}},
                "texto":{"type":"text"},
                "ip_comentarista":{"type":"ip"} # mapeando como IP
            }
        },
        'bytes_mp4': {'type': 'binary'}, # mapeando como binário
        'musica_tocada_radio': {'type': 'boolean','null_value':None} # mapeando como booleano, com valor padrão null
    }
}

Vários campos de texto foram mapeados corretamente como **texto** ou **palavra-chave**, otimizando o uso de memória.

O campo **"ano"** foi interpretado como um campo de data, porém restrito ao formato **"yyyy"** (somente o ano).

Além disso:

- O campo **"bytes_mp4"** foi mapeado como **binário**.
- O campo **"ip_comentarista"** foi mapeado como tipo **IP**.
- O campo **"comentarios"** foi mapeado como tipo **nested** (aninhado), o que permite consultas estruturadas dentro de arrays de objetos.

Mais detalhes sobre esse mapeamento podem ser encontrados no notebook **[Mapeamento - JSON]**.


In [30]:
NOVO_INDICE_MAPPING_MUSICA = 'novo_mapeamento_musica'
if client.indices.exists(index=NOVO_INDICE_MAPPING_MUSICA):
    client.indices.delete(index=NOVO_INDICE_MAPPING_MUSICA)
client.indices.create(index=NOVO_INDICE_MAPPING_MUSICA, mappings=novo_mapeamento)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'novo_mapeamento_musica'})

In [31]:
#Salvando novamente o objeto com o novo mapeamento
client.index(index=NOVO_INDICE_MAPPING_MUSICA, document=letra_musica)

ObjectApiResponse({'_index': 'novo_mapeamento_musica', '_id': '5bOTS5cBUW64_IbY3R09', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [22]:
#verificando o novo mapeamento
novo_mapping_musica = client.indices.get_mapping(index=NOVO_INDICE_MAPPING_MUSICA)

In [32]:
novo_mapping_musica.body

{'novo_mapeamento_musica': {'mappings': {'properties': {'administracao': {'type': 'text'},
    'ano': {'type': 'date', 'format': 'yyyy'},
    'autor': {'type': 'keyword'},
    'bytes_mp4': {'type': 'binary'},
    'comentarios': {'type': 'nested',
     'properties': {'autor': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword'}}},
      'data': {'type': 'date', 'format': 'yyyy-MM-dd'},
      'ip_comentarista': {'type': 'ip'},
      'texto': {'type': 'text'},
      'titulo': {'type': 'text', 'fields': {'keyword': {'type': 'keyword'}}}}},
    'compositor': {'type': 'keyword'},
    'editor': {'type': 'text'},
    'extracao': {'type': 'date', 'format': 'yyyy-MM-dd'},
    'extracao_exata': {'type': 'date', 'format': 'yyyy-MM-dd'},
    'extracao_mes': {'type': 'date', 'format': 'yyyy-MM'},
    'genero': {'type': 'keyword'},
    'letra': {'type': 'text'},
    'musica_tocada_radio': {'type': 'boolean'},
    'pontuacao_letras': {'type': 'float'},
    'pontuacao_vagalume': {'type': '

In [33]:
#Refazendo a pesquisa por autor
search_query = {
    "query": {
        "match": {
            "autor": "Charlie"
        }
    }
}
response = client.search(index=NOVO_INDICE_MAPPING_MUSICA, body=search_query)

In [34]:
response.body

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

Como o "autor" foi mapeado como palavra-chave (keyword), somente termo exato funcionará na pesquisa

In [37]:
#Refazendo a pesquisa por autor
search_query_keyword = {
    "query": {
        "match": {
            "autor": "Charlie Brown Jr"
        }
    }
}   

response = client.search(index=NOVO_INDICE_MAPPING_MUSICA, body=search_query_keyword)

In [38]:
response.body

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.2876821,
  'hits': [{'_index': 'novo_mapeamento_musica',
    '_id': '5bOTS5cBUW64_IbY3R09',
    '_score': 0.2876821,
    '_source': {'titulo': 'Meu Novo Mundo',
     'autor': 'Charlie Brown Jr',
     'letra': 'Como se o silêncio dissesse tudo  \n  Um sentimento bom que me leva pra outro mundo  \n  A vontade de te ver já é maior que tudo  \n  Não existem distâncias no meu novo mundo  \n    \n  Tipo coisas da sétima arte  \n  Aconteceu sem que eu imaginasse  \n    \n  Sonho de consumo cantar na sua festa  \n  Vem dançar comigo  \n  Aproveita e me sequestra  \n  Amor vagabundo, intenso ou muita pressa  \n  Não sei como termina mas sei como começa  \n    \n  Fiz essa canção pra dizer algumas coisas  \n  Cuidado com o destino  \n  Ele brinca com as pessoas  \n  Tipo uma foto com sorriso inocente  \n  Mas a vida tinha um pl

### Salvando um novo objeto

Quando um novo objeto é salvo em um indice com mapeamento, o elastisearch valida a compartibilidade. 
Se o objeto tiver alguma chave incompatível , o objeto não é salvo

In [39]:
#Tentando salvar um documento com o campo "ano" no formato errado

letra_musica_errada = {
    "ano":"2025-06-03",  # Formato errado, deveria ser apenas o ano
}

client.index(index=NOVO_INDICE_MAPPING_MUSICA, document=letra_musica_errada)

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', "failed to parse field [ano] of type [date] in document with id '5rOYS5cBUW64_IbYZh0j'. Preview of field's value: '2025-06-03'")